<a href="https://colab.research.google.com/github/pavang2579/Delivery-duration-Prediction/blob/main/DoorDash_Delivery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Doordash Delivery duration prediction


In [15]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as matplot
import seaborn as sns
import csv

In [19]:
# read data
# path = "C:\\Users\\pavan\\Desktop\\projects\\Doordash delivery\\datasets\\historical_data.csv"
data = pd.read_csv('/content/historical_data.csv')

In [20]:
# Summary of the data

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63448 entries, 0 to 63447
Data columns (total 16 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   market_id                                     63123 non-null  float64
 1   created_at                                    63448 non-null  object 
 2   actual_delivery_time                          63446 non-null  object 
 3   store_id                                      63448 non-null  int64  
 4   store_primary_category                        61834 non-null  object 
 5   order_protocol                                63131 non-null  float64
 6   total_items                                   63448 non-null  int64  
 7   subtotal                                      63448 non-null  int64  
 8   num_distinct_items                            63448 non-null  int64  
 9   min_item_price                                63448 non-null 

In [25]:
data['created_at'] = pd.to_datetime(data['created_at'])
data['actual_delivery_time'] = pd.to_datetime(data['actual_delivery_time'])


In [26]:
#create target variable for regression
from datetime import datetime 
data['actual_total_deliver_duration'] = data['actual_delivery_time']-data['created_at']


In [28]:
#new variable creation busydashers ratio
data['busy_dashers_ration'] = data['total_busy_dashers']/data['total_onshift_dashers']

In [30]:
data['non_preperation_time'] = data['estimated_store_to_consumer_driving_duration'] + data['estimated_order_place_duration']

In [33]:
#create dummies fro market id
market_id_dummies = pd.get_dummies(data['market_id'])
market_id_dummies = market_id_dummies.add_prefix('market_id')
market_id_dummies.head()


,market_id1.0,market_id2.0,market_id3.0,market_id4.0,market_id5.0,market_id6.0
0,1,0,0,0,0,0
1,0,1,0,0,0,0
2,0,0,1,0,0,0
3,0,0,1,0,0,0
4,0,0,1,0,0,0


In [34]:
#create dummies fro order protocol
order_protocol_dummies = pd.get_dummies(data['order_protocol'])
order_protocol_dummies = order_protocol_dummies.add_prefix('order_protocol')
order_protocol_dummies.head()

,order_protocol1.0,order_protocol2.0,order_protocol3.0,order_protocol4.0,order_protocol5.0,order_protocol6.0,order_protocol7.0
0,1,0,0,0,0,0,0
1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0
3,1,0,0,0,0,0,0
4,1,0,0,0,0,0,0


In [43]:
#create dictionary with the most repeated value for each store to fill the null rows 
store_id_unique = data['store_id'].unique().tolist()
store_id_and_category = {store_id:data[data.store_id == store_id].store_primary_category.mode() for store_id in store_id_unique}



{1845: 0    american
 Name: store_primary_category, dtype: object,
 5477: 0     indian
 1    mexican
 Name: store_primary_category, dtype: object,
 2841: 0    italian
 Name: store_primary_category, dtype: object,
 4139: 0    mexican
 Name: store_primary_category, dtype: object,
 5058: 0    italian
 Name: store_primary_category, dtype: object,
 4149: 0    sandwich
 1        thai
 Name: store_primary_category, dtype: object,
 3201: 0    cafe
 Name: store_primary_category, dtype: object,
 5054: 0    italian
 Name: store_primary_category, dtype: object,
 2004: 0    american
 Name: store_primary_category, dtype: object,
 1984: 0    chinese
 Name: store_primary_category, dtype: object,
 4905: 0    mexican
 Name: store_primary_category, dtype: object,
 1698: 0    sandwich
 Name: store_primary_category, dtype: object,
 1936: 0    singaporean
 Name: store_primary_category, dtype: object,
 6015: 0    burger
 Name: store_primary_category, dtype: object,
 976: 0    breakfast
 Name: store_primary_c

In [44]:
def fill(store_id):
  try:
    return store_id_and_category[store_id].value[0]
  except:
    return np.nan

# fill null value
data['nan_free_store_primary_category'] = data.store_id.apply(fill)